In [11]:
%matplotlib inline

import pandas as pd
import category_encoders as ce
from sklearn.preprocessing import LabelBinarizer, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [12]:
train = pd.read_csv('./data/rawdata/application_train.csv.zip',compression='zip')
test = pd.read_csv('./data/rawdata/application_test.csv.zip',compression='zip')

In [13]:
train.shape
train.head()

(307511, 122)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# train.TARGET.groupby(train.AMT_REQ_CREDIT_BUREAU_YEAR).agg(['mean','count'])
# train.AMT_REQ_CREDIT_BUREAU_YEAR.value_counts(dropna=False)
# [1 if b == 'Family' else 0 for b in ['Unaccompanied', 'Family', 'Spouse, partner', 'Children', 'Other_A','Other_B', 'Group of people']]

# import category_encoders as ce
# encoder = ce.OneHotEncoder(cols=['FLAG_DOCUMENT_5','FLAG_OWN_CAR'])
# encoder.fit(train[['FLAG_DOCUMENT_5','FLAG_OWN_CAR']])
# encoder.transform(train[['FLAG_DOCUMENT_5','FLAG_OWN_CAR']])

In [15]:
feature_dict = {'NAME_CONTRACT_TYPE':'category_features', 'CODE_GENDER':'category_features', 'FLAG_OWN_CAR':'boolean_features', 'FLAG_OWN_REALTY':'boolean_features', 'CNT_CHILDREN':'count_features', 'AMT_INCOME_TOTAL':'amount_feautres', 'AMT_CREDIT':'amount_feautres', 'AMT_ANNUITY':'amount_feautres', 'AMT_GOODS_PRICE':'amount_feautres', 'NAME_TYPE_SUITE':'category_features', 'NAME_INCOME_TYPE':'category_features', 'NAME_EDUCATION_TYPE':'category_features', 'NAME_FAMILY_STATUS':'category_features', 'NAME_HOUSING_TYPE':'category_features', 'REGION_POPULATION_RELATIVE':'normal_features', 'DAYS_BIRTH':'amount_feautres', 'DAYS_EMPLOYED':'amount_feautres', 'DAYS_REGISTRATION':'amount_feautres', 'DAYS_ID_PUBLISH':'amount_feautres', 'OWN_CAR_AGE':'amount_feautres', 'FLAG_MOBIL':'boolean_features', 'FLAG_EMP_PHONE':'boolean_features', 'FLAG_WORK_PHONE':'boolean_features', 'FLAG_CONT_MOBILE':'boolean_features', 'FLAG_PHONE':'boolean_features', 'FLAG_EMAIL':'boolean_features', 'OCCUPATION_TYPE':'category_features', 'CNT_FAM_MEMBERS':'count_features', 'REGION_RATING_CLIENT':'category_features', 'REGION_RATING_CLIENT_W_CITY':'category_features', 'WEEKDAY_APPR_PROCESS_START':'category_features', 'HOUR_APPR_PROCESS_START':'category_features', 'REG_REGION_NOT_LIVE_REGION':'boolean_features', 'REG_REGION_NOT_WORK_REGION':'boolean_features', 'LIVE_REGION_NOT_WORK_REGION':'boolean_features', 'REG_CITY_NOT_LIVE_CITY':'boolean_features', 'REG_CITY_NOT_WORK_CITY':'boolean_features', 'LIVE_CITY_NOT_WORK_CITY':'boolean_features', 'ORGANIZATION_TYPE':'category_features', 'EXT_SOURCE_1':'normal_features', 'EXT_SOURCE_2':'normal_features', 'EXT_SOURCE_3':'normal_features', 'APARTMENTS_AVG':'normal_features', 'BASEMENTAREA_AVG':'normal_features', 'YEARS_BEGINEXPLUATATION_AVG':'normal_features', 'YEARS_BUILD_AVG':'normal_features', 'COMMONAREA_AVG':'normal_features', 'ELEVATORS_AVG':'normal_features', 'ENTRANCES_AVG':'normal_features', 'FLOORSMAX_AVG':'normal_features', 'FLOORSMIN_AVG':'normal_features', 'LANDAREA_AVG':'normal_features', 'LIVINGAPARTMENTS_AVG':'normal_features', 'LIVINGAREA_AVG':'normal_features', 'NONLIVINGAPARTMENTS_AVG':'normal_features', 'NONLIVINGAREA_AVG':'normal_features', 'APARTMENTS_MODE':'normal_features', 'BASEMENTAREA_MODE':'normal_features', 'YEARS_BEGINEXPLUATATION_MODE':'normal_features', 'YEARS_BUILD_MODE':'normal_features', 'COMMONAREA_MODE':'normal_features', 'ELEVATORS_MODE':'normal_features', 'ENTRANCES_MODE':'normal_features', 'FLOORSMAX_MODE':'normal_features', 'FLOORSMIN_MODE':'normal_features', 'LANDAREA_MODE':'normal_features', 'LIVINGAPARTMENTS_MODE':'normal_features', 'LIVINGAREA_MODE':'normal_features', 'NONLIVINGAPARTMENTS_MODE':'normal_features', 'NONLIVINGAREA_MODE':'normal_features', 'APARTMENTS_MEDI':'normal_features', 'BASEMENTAREA_MEDI':'normal_features', 'YEARS_BEGINEXPLUATATION_MEDI':'normal_features', 'YEARS_BUILD_MEDI':'normal_features', 'COMMONAREA_MEDI':'normal_features', 'ELEVATORS_MEDI':'normal_features', 'ENTRANCES_MEDI':'normal_features', 'FLOORSMAX_MEDI':'normal_features', 'FLOORSMIN_MEDI':'normal_features', 'LANDAREA_MEDI':'normal_features', 'LIVINGAPARTMENTS_MEDI':'normal_features', 'LIVINGAREA_MEDI':'normal_features', 'NONLIVINGAPARTMENTS_MEDI':'normal_features', 'NONLIVINGAREA_MEDI':'normal_features', 'FONDKAPREMONT_MODE':'category_features', 'HOUSETYPE_MODE':'category_features', 'TOTALAREA_MODE':'normal_features', 'WALLSMATERIAL_MODE':'category_features', 'EMERGENCYSTATE_MODE':'category_features', 'OBS_30_CNT_SOCIAL_CIRCLE':'count_features', 'DEF_30_CNT_SOCIAL_CIRCLE':'count_features', 'OBS_60_CNT_SOCIAL_CIRCLE':'count_features', 'DEF_60_CNT_SOCIAL_CIRCLE':'count_features', 'DAYS_LAST_PHONE_CHANGE':'count_features', 'FLAG_DOCUMENT_2':'boolean_features', 'FLAG_DOCUMENT_3':'boolean_features', 'FLAG_DOCUMENT_4':'boolean_features', 'FLAG_DOCUMENT_5':'boolean_features', 'FLAG_DOCUMENT_6':'boolean_features', 'FLAG_DOCUMENT_7':'boolean_features', 'FLAG_DOCUMENT_8':'boolean_features', 'FLAG_DOCUMENT_9':'boolean_features', 'FLAG_DOCUMENT_10':'boolean_features', 'FLAG_DOCUMENT_11':'boolean_features', 'FLAG_DOCUMENT_12':'boolean_features', 'FLAG_DOCUMENT_13':'boolean_features', 'FLAG_DOCUMENT_14':'boolean_features', 'FLAG_DOCUMENT_15':'boolean_features', 'FLAG_DOCUMENT_16':'boolean_features', 'FLAG_DOCUMENT_17':'boolean_features', 'FLAG_DOCUMENT_18':'boolean_features', 'FLAG_DOCUMENT_19':'boolean_features', 'FLAG_DOCUMENT_20':'boolean_features', 'FLAG_DOCUMENT_21':'boolean_features', 'AMT_REQ_CREDIT_BUREAU_HOUR':'count_features', 'AMT_REQ_CREDIT_BUREAU_DAY':'count_features', 'AMT_REQ_CREDIT_BUREAU_WEEK':'count_features', 'AMT_REQ_CREDIT_BUREAU_MON':'count_features', 'AMT_REQ_CREDIT_BUREAU_QRT':'count_features', 'AMT_REQ_CREDIT_BUREAU_YEAR':'count_features'}

In [16]:
onehot_encoder = ce.LeaveOneOutEncoder(cols = ['FLAG_DOCUMENT_20'])
onehot_encoder.fit(train[['FLAG_DOCUMENT_20']], train.TARGET)
onehot_encoder.transform(train[['FLAG_DOCUMENT_20']]).head()

taget_encoder = ce.TargetEncoder(cols=['FLAG_DOCUMENT_20'])
taget_encoder.fit(train[['FLAG_DOCUMENT_20']], train.TARGET)
taget_encoder.transform(train[['FLAG_DOCUMENT_20']]).head()


LeaveOneOutEncoder(cols=['FLAG_DOCUMENT_20'], drop_invariant=False,
          handle_unknown='impute', impute_missing=True, random_state=None,
          randomized=False, return_df=True, sigma=0.05, verbose=0)

,FLAG_DOCUMENT_20
0,0.080727
1,0.080727
2,0.080727
3,0.080727
4,0.080727


TargetEncoder(cols=['FLAG_DOCUMENT_20'], drop_invariant=False,
       handle_unknown='impute', impute_missing=True, min_samples_leaf=1,
       return_df=True, smoothing=1, verbose=0)

,FLAG_DOCUMENT_20
0,0.080727
1,0.080727
2,0.080727
3,0.080727
4,0.080727


In [17]:
special_features = ['OWN_CAR_AGE','CNT_CHILDREN','CNT_FAM_MEMBERS',
                    'HOUR_APPR_PROCESS_START','WEEKDAY_APPR_PROCESS_START',
                    'AMT_REQ_CREDIT_BUREAU_HOUR','AMT_REQ_CREDIT_BUREAU_DAY',
                    'AMT_REQ_CREDIT_BUREAU_WEEK','AMT_REQ_CREDIT_BUREAU_MON',
                    'AMT_REQ_CREDIT_BUREAU_QRT','AMT_REQ_CREDIT_BUREAU_YEAR'
                   ]

In [18]:
category_features = [f for f in feature_dict if feature_dict[f] == 'category_features' ]
boolean_features = [f for f in feature_dict if feature_dict[f] == 'boolean_features' ]
amount_feautres = [f for f in feature_dict if feature_dict[f] == 'amount_feautres' ]
normal_features = [f for f in feature_dict if feature_dict[f] == 'normal_features' ]
count_features = [f for f in feature_dict if feature_dict[f] == 'count_features' ]

In [19]:
taget_encoder = ce.TargetEncoder(cols=category_features + boolean_features)
taget_encoder.fit(train[category_features + boolean_features], train.TARGET)

onehot_encoder = ce.OneHotEncoder(cols = category_features + boolean_features)
onehot_encoder.fit(train[category_features + boolean_features])

ordinal_encoder = ce.OrdinalEncoder(cols = category_features)
ordinal_encoder.fit(train[category_features])

normalize = StandardScaler().fit(train[amount_feautres].fillna(0))

X_train = pd.concat([
    train[normal_features].fillna(0),
    taget_encoder.transform(train[category_features + boolean_features]),
    onehot_encoder.transform(train[category_features + boolean_features]),
    ordinal_encoder.transform(train[category_features]),
    pd.DataFrame(normalize.transform(train[amount_feautres].fillna(0)), columns = amount_feautres)
], axis = 1)
y_train = train.TARGET


X_test = pd.concat([
    test[normal_features].fillna(0),
    taget_encoder.transform(test[category_features + boolean_features]),
    onehot_encoder.transform(test[category_features + boolean_features]),
    ordinal_encoder.transform(test[category_features]),
    pd.DataFrame(normalize.transform(test[amount_feautres].fillna(0)), columns = amount_feautres)
], axis = 1)

TargetEncoder(cols=['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START', 'ORGANIZATION_TYPE', ... 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21'],
       drop_invariant=False, handle_unknown='impute', impute_missing=True,
       min_samples_leaf=1, return_df=True, smoothing=1, verbose=0)

OneHotEncoder(cols=['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START', 'ORGANIZATION_TYPE', ... 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21'],
       drop_invariant=False, handle_unknown='impute', impute_missing=True,
       return_df=True, verbose=0)

OrdinalEncoder(cols=['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE'],
        drop_invariant=False, handle_unknown='impute', impute_missing=True,
        mapping=[{'col': 'NAME_CONTRACT_TYPE', 'mapping': [('Cash loans', 0), ('Revolving loans', 1)]}, {'col': 'CODE_GENDER', 'mapping': [('M', 0), ('F', 1), ('XNA', 2)]}, {'col': 'NAME_TYPE_SUITE', 'mapping': [('Unaccompanied', 0), ('Family', 1), ('Spouse, partner', 2), ('Children', 3), ('Other_A', 4), (n..., ('Monolithic', 7)]}, {'col': 'EMERGENCYSTATE_MODE', 'mapping': [('No', 0), (nan, 1), ('Yes', 2)]}],
        return_df=True, verbose=0)

In [20]:
X_train.shape
X_train.head()

(307511, 415)

,REGION_POPULATION_RELATIVE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,...,EMERGENCYSTATE_MODE,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE
0,0.018801,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,...,0,0.142129,-0.478095,-0.166065,-0.505662,1.506880,-0.456215,0.379837,0.579154,-0.455284
1,0.003541,0.311267,0.622246,0.000000,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,...,0,0.426792,1.725450,0.592710,1.600480,-0.166821,-0.460115,1.078697,1.790855,-0.455284
2,0.010032,0.000000,0.555912,0.729567,0.0000,0.0000,0.0000,0.0000,0.0000,0.00,...,-1,-0.427196,-1.152888,-1.404507,-1.090025,-0.689509,-0.453299,0.206116,0.306869,2.430559
3,0.008019,0.000000,0.650442,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.00,...,-1,-0.142533,-0.711430,0.177929,-0.651753,-0.680114,-0.473217,-1.375829,0.369143,-0.455284
4,0.028663,0.000000,0.322738,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.00,...,-1,-0.199466,-0.213734,-0.361658,-0.067390,-0.892535,-0.473210,0.191639,-0.307263,-0.455284


In [185]:
lr = LogisticRegression(C = 0.1, penalty = 'l1', random_state = 42)
lr.fit(X_train, y_train)
roc_auc_score(y_train, lr.predict_proba(X_train)[:,1])

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

0.72829770898856683

In [186]:
test.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [188]:
sub = pd.concat([
    test.SK_ID_CURR,
    pd.DataFrame({'TARGET':lr.predict_proba(X_test)[:,1]})
], axis = 1)


In [190]:
sub.head()
sub.to_csv('./data/submission/first_20180520.csv', index = False)

,SK_ID_CURR,TARGET
0,100001,0.037413
1,100005,0.131868
2,100013,0.037839
3,100028,0.030422
4,100038,0.156930


In [25]:
pd.concat([X_train,pd.DataFrame(y_train, columns = ['TARGET'])], axis = 1).to_csv('./data/input/train.csv', index = False)
# y_train.to_csv('./data/input/y_train.csv', index = False)
X_test.to_csv('./data/input/test.csv', index = False)
